In [1]:
from sys import path
path.append(r'C:\Users\Izogie\Desktop\Folders\Projects\Python\KB Chat\src')
from modules.SourceManager import SourceManager
from modules.VectorDBManager import VectorDBManager
from Pipeline import RAGPipeline

<frozen importlib._bootstrap>:488: DeprecationWarning: Type google._upb._message.MessageMapContainer uses PyType_Spec with a metaclass that has custom tp_new. This is deprecated and will no longer be allowed in Python 3.14.
<frozen importlib._bootstrap>:488: DeprecationWarning: Type google._upb._message.ScalarMapContainer uses PyType_Spec with a metaclass that has custom tp_new. This is deprecated and will no longer be allowed in Python 3.14.
c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Testing VectorDBManager

In [2]:
data_manager = SourceManager()

In [3]:
page_titles = [
    "Kaladin",
    "Pits of Hathsin",
    "Allomancy",
    "Cosmere",
    "Honor's Perpendicularity",
    "Cephandrius",
    "Hoid",
    "Odium",
    "Realmatic Theory",
    "Knights Radiant",
    "Kelsier",
    "Stormfather",
    "Cognitive Shadow",
    "Dalinar Kholin",
    "Surgebinder",
    "Shadesmar",
    "Awakening",
    "Tanavast",
    "Urithiru",
    "Fused"
]


In [4]:
# data = data_manager.prep_data_graph(page_titles)
data = data_manager.load_json("processed_articles.jsonl")

In [5]:
vectorManager = VectorDBManager()

c:\Python312\Lib\site-packages\posthog\client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())
c:\Python312\Lib\site-packages\posthog\client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())


In [6]:
vectorManager.ingest_articles(data)

c:\Python312\Lib\site-packages\posthog\request.py:40: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  body["sentAt"] = datetime.utcnow().replace(tzinfo=tzutc()).isoformat()
c:\Python312\Lib\site-packages\posthog\client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())


In [7]:
# from langchain.chains.query_constructor.base import AttributeInfo
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.retrievers.self_query.chroma import ChromaTranslator

vectorManager._init_langchaindb()
vectorManager._init_metadata_field_info()
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro"
)
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore=vectorManager.langdb,
    document_contents=vectorManager.doc_content_description,
    metadata_field_info=vectorManager.metadata_field_info,
    structured_query_translator=ChromaTranslator()
)

c:\Python312\Lib\site-packages\posthog\request.py:40: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  body["sentAt"] = datetime.utcnow().replace(tzinfo=tzutc()).isoformat()
c:\Python312\Lib\site-packages\posthog\client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())


In [8]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
compressor = FlashrankRerank()
rerank_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever)

c:\Python312\Lib\site-packages\posthog\request.py:40: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  body["sentAt"] = datetime.utcnow().replace(tzinfo=tzutc()).isoformat()


In [9]:
from langchain_google_genai import GoogleGenerativeAI
from google.generativeai import configure
from os import environ
configure(api_key=environ["GOOGLE_API_KEY"])
gemini = GoogleGenerativeAI(model="gemini-1.5-flash")

In [21]:
query = "What is the difference between the fused and the singers?"

In [22]:
gemini_docs = rerank_retriever.invoke(query)
print(gemini_docs)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


[Document(metadata={'id': 2, 'relevance_score': 0.99887574}, page_content='The Fused are the souls of ancient singers that are able to be reborn by possessing the bodies of living singers. They are considered to be higher in singer society than Regals and common singers. They are Surgebinders; however, unlike the Knights Radiant, there are only nine "brands" of them, rather than ten, and each brand has access to a single Surge, with none accessing Adhesion.'), Document(metadata={'id': 3, 'relevance_score': 0.99887574}, page_content='The Fused are the souls of ancient singers that are able to be reborn by possessing the bodies of living singers. They are considered to be higher in singer society than Regals and common singers. They are Surgebinders; however, unlike the Knights Radiant, there are only nine "brands" of them, rather than ten, and each brand has access to a single Surge, with none accessing Adhesion.'), Document(metadata={'id': 0, 'relevance_score': 0.99875003}, page_conten

In [23]:

prompt = f"""
Use the below context to assist in answering this question: {query}

context
```
{gemini_docs}
```
"""
gem_response = gemini.invoke(prompt)

In [24]:
print(gem_response)

The provided context describes two types of singers: **Fused** and **common singers**. Here's the breakdown of their differences:

* **Fused:**
    * **Origin:** Souls of ancient singers reborn by possessing the bodies of living singers.
    * **Status:** Higher in singer society than Regals and common singers.
    * **Powers:** Surgebinders with access to nine "brands," each wielding a single Surge (no Adhesion).
    * **Hierarchy:** Have a complex political and social hierarchy within their ranks, with different brands, reputations, and levels of sanity impacting their standing.
    * **Attitude towards humans:** Generally harbor hatred and contempt for humans.

* **Common singers:**
    * **Origin:** Ordinary singers who are not Fused.
    * **Status:** Lower than Fused and Regals in singer society.
    * **Powers:** Some common singers are chosen to be elevated and become Regals, granting them forms of power.
    * **Hierarchy:**  The context mentions a hierarchy among common singe

# RAG Pipeline v1 class test

In [2]:
pipe = RAGPipeline()

c:\Python312\Lib\site-packages\posthog\client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())
c:\Python312\Lib\site-packages\posthog\client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())


In [3]:
# pipe.load_processed_pages()
pipe.init_retriever()
pipe.init_compressor()

c:\Python312\Lib\site-packages\posthog\request.py:40: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  body["sentAt"] = datetime.utcnow().replace(tzinfo=tzutc()).isoformat()
c:\Python312\Lib\site-packages\posthog\client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())


In [4]:
query = "What is the difference between the fused and the singers?"

In [5]:
response = pipe.perform_rag(query)

c:\Python312\Lib\site-packages\posthog\request.py:40: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  body["sentAt"] = datetime.utcnow().replace(tzinfo=tzutc()).isoformat()
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
c:\Python312\Lib\site-packages\posthog\client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())
c:\Python312\Lib\site-packages\posthog\request.py:40: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  body["sentAt"


The main difference between the Fused and the singers is that **singers are a subset of the Fused**. 

Here's the breakdown:

* **Fused:** This is the overarching category, encompassing beings who have been "fused" with shards of a powerful entity called Odium. They are generally hostile to humans and have a complex social hierarchy.
* **Singers:** These are a specific type of Fused who possess certain abilities and are more aligned with Odium's will. They are considered higher in status than regular humans. 
* **Regals:** These are singers who have been elevated to positions of power.
* **Voices:** These are singers who represent and manage the affairs of important Fused. They are often respected more than even Regals.

The text highlights that even within the singers, there are distinctions of status and authority based on factors like their brand, reputation, history, and sanity.

Sources:
1 - The Fused have an extremely complex political and social hierarchy even though their acti

In [6]:
print(response)

('The main difference between the Fused and the singers is that **singers are a subset of the Fused**. \n\nHere\'s the breakdown:\n\n* **Fused:** This is the overarching category, encompassing beings who have been "fused" with shards of a powerful entity called Odium. They are generally hostile to humans and have a complex social hierarchy.\n* **Singers:** These are a specific type of Fused who possess certain abilities and are more aligned with Odium\'s will. They are considered higher in status than regular humans. \n* **Regals:** These are singers who have been elevated to positions of power.\n* **Voices:** These are singers who represent and manage the affairs of important Fused. They are often respected more than even Regals.\n\nThe text highlights that even within the singers, there are distinctions of status and authority based on factors like their brand, reputation, history, and sanity.', [Document(metadata={'id': 3, 'relevance_score': 0.9986379}, page_content='The Fused have 

# RAG Pipeline v2 class test

In [2]:
pipe = RAGPipeline()

c:\Python312\Lib\site-packages\posthog\client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())
c:\Python312\Lib\site-packages\posthog\request.py:40: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  body["sentAt"] = datetime.utcnow().replace(tzinfo=tzutc()).isoformat()
c:\Python312\Lib\site-packages\posthog\client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())
c:\Python312\Lib\site-packages\posthog\clien

In [3]:
pipe.retriever.load_processed()

In [5]:
pipe.perform_rag("How do the Alethi armies sustain themselves?")


The provided text focuses on Urithiru, a tower city in the world of Roshar, and does not mention the Alethi armies' sustenance. Therefore, the context does not provide an answer to the question of how the Alethi armies sustain themselves. 

To answer this question, you would need to consult other sources about the Alethi culture and military in the Cosmere, the fictional universe in which the story takes place.

Sources:
1 - Urithiru -  Culture 

Though massively important in the past, prior to the coming of the Everstorm Urithiru was all but unknown to Vorins who didn't study the Knights Radiant, with Shallan being unfamiliar even with its name. This could be the result of the Hierocracy working to suppress all knowledge of Radiants, similar to how they discredited Surgebinding.

In contrast, Shinovar is familiar with Urithiru, at least insofar as the traditions surrounding it are concerned. According to the Shin, the stones in Urithiru are unhallowed, making them fine to walk on, as

("The provided text focuses on Urithiru, a tower city in the world of Roshar, and does not mention the Alethi armies' sustenance. Therefore, the context does not provide an answer to the question of how the Alethi armies sustain themselves. \n\nTo answer this question, you would need to consult other sources about the Alethi culture and military in the Cosmere, the fictional universe in which the story takes place.",
 [Document(metadata={'heading': 'Culture', 'order': 19, 'parent_article': 'Urithiru', 'keywords': '', 'relevance_score': 0.99491054}, page_content="Urithiru -  Culture \n\nThough massively important in the past, prior to the coming of the Everstorm Urithiru was all but unknown to Vorins who didn't study the Knights Radiant, with Shallan being unfamiliar even with its name. This could be the result of the Hierocracy working to suppress all knowledge of Radiants, similar to how they discredited Surgebinding.\n\nIn contrast, Shinovar is familiar with Urithiru, at least insofa